In [1]:
%matplotlib inline
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
import plotly.graph_objects as go
import statsmodels.api         as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score
# always make it pretty 
plt.style.use('ggplot')
from practicum_utils import get_loggi_files, global_connect, run_query, explained_time, careful_query

In [2]:
db = global_connect()

In [35]:
run_query("""
CREATE TABLE av2it_oozma (
    availability_id    VARCHAR(32) NOT NULL,
    itinerary_id       VARCHAR(32) NOT NULL
);

ALTER TABLE av2it_oozma ADD PRIMARY KEY (availability_id, itinerary_id);
""", df=False)

In [36]:
df_it = careful_query("""select * from itinerary""")

In [37]:
for i in range(len(df_it)):
    df_av = careful_query("""
    select * from availability
    where driver_id = '{}'
      and sent_f >= '{}'
      and sent_f <= '{}'
    """.format(df_it.iloc[i].driver_id,
               df_it.iloc[i].accepted_time,
               df_it.iloc[i].finished_time))
    
    insert = 'INSERT INTO av2it_oozma  (availability_id, itinerary_id) VALUES '
    if len(df_av) > 0:
        for j in range(len(df_av)):
            insert += "('{}', '{}'),".format(df_av.iloc[j]['id'], df_it.iloc[i]['itinerary_id'])
        insert = insert[0:-1]
        print('{:05d} - Inserting {} slots for driver_id {}'.format(i, len(df_av), df_it.iloc[i].driver_id))
        r = run_query(insert, df=False)
    else:
        print('{:05d} - No available slots found for driver_id {} between {} and {}'.format(
            i, df_it.iloc[i].driver_id,
               df_it.iloc[i].accepted_time,
               df_it.iloc[i].finished_time))


00000 - Inserting 11 slots for driver_id f27a3ee8abdb98f6a6162c67e1c4b9da
00001 - No available slots found slots for driver_id a6eaa784d041ef2476968e78cf26463a between 2020-03-04 17:00:08 and 2020-03-04 17:00:34
00002 - No available slots found slots for driver_id abae2bc074c8b4af3a7272b12ba66510 between 2020-02-26 15:50:16 and 2020-02-26 15:52:33
00003 - Inserting 25 slots for driver_id 4a4fc0609d5e034e8e35adf86ed4df9b
00004 - Inserting 13 slots for driver_id bf3f0c443de78920f7e076e9dc8e4a73
00005 - Inserting 4 slots for driver_id e8030e78e0bffd714610529c10e1be5d
00006 - Inserting 4 slots for driver_id abae2bc074c8b4af3a7272b12ba66510
00007 - Inserting 23 slots for driver_id 75e89a14d6ee1fe153d9062a75574ece
00008 - Inserting 21 slots for driver_id 51e3fb34a0495f9e33c2c141aef74442
00009 - Inserting 6 slots for driver_id 5e010c995984d99e3c8dfc787e682508
00010 - Inserting 1 slots for driver_id abae2bc074c8b4af3a7272b12ba66510
00011 - Inserting 1 slots for driver_id a6eaa784d041ef2476968e

00113 - Inserting 29 slots for driver_id ae1649b07da8cf8ce38cb601def53b66
00114 - Inserting 40 slots for driver_id 5db30d43f3791ae82e8f09070647e4cb
00115 - Inserting 15 slots for driver_id c6351f9aebae5a4c1cd1a33dded791f4
00116 - Inserting 1 slots for driver_id a6eaa784d041ef2476968e78cf26463a
00117 - Inserting 11 slots for driver_id 0d7aaef9dc7b2dba37abe54c165adc5b
00118 - Inserting 13 slots for driver_id fae272b321c99415bc913513053c5fd2
00119 - Inserting 28 slots for driver_id fae272b321c99415bc913513053c5fd2
00120 - No available slots found slots for driver_id 12ea2130f599973251826ea003793240 between 2020-03-25 17:23:50 and 2020-03-25 17:24:27
00121 - Inserting 42 slots for driver_id f878386eac8037d383f404692cfc3b2c
00122 - Inserting 2 slots for driver_id fae272b321c99415bc913513053c5fd2
00123 - Inserting 2 slots for driver_id a9a2ac00c1fe04c8236a1152c11e444b
00124 - Inserting 139 slots for driver_id eb8d2bef11722a414e17cbc860beda66
00125 - Inserting 4 slots for driver_id 8c3916559e

00218 - Inserting 34 slots for driver_id 478da8aee42a6a17b7504d2c9fd75ed2
00219 - Inserting 7 slots for driver_id 4416d590c9179a08081ba762d365b799
00220 - Inserting 67 slots for driver_id 7e62eec6f9c8ec6c227766dbe18d12e5
00221 - Inserting 11 slots for driver_id 694e48a4102125236ae8fd51aebaffb1
00222 - Inserting 11 slots for driver_id 03c85ce82febe2cbc45a639ae8a1432a
00223 - No available slots found slots for driver_id abae2bc074c8b4af3a7272b12ba66510 between 2020-03-25 17:13:58 and 2020-03-25 17:14:45
00224 - Inserting 22 slots for driver_id f2129ed850d55c21e4119ced9f4cf815
00225 - Inserting 2 slots for driver_id 987ff3e8d7e276cd512363037c011993
00226 - Inserting 2 slots for driver_id fae272b321c99415bc913513053c5fd2
00227 - Inserting 3 slots for driver_id dc716906fcfcff036b06123fed559d89
00228 - Inserting 29 slots for driver_id 878f28b136310cacb1405a26a29afaf5
00229 - No available slots found slots for driver_id fdc998e2466a148f45587e2281a26325 between 2020-03-25 16:48:56 and 2020-03-

00317 - Inserting 2 slots for driver_id 0052e0a5b243cb0855827577443dcf97
00318 - Inserting 8 slots for driver_id 5e6d111d999b0afdbfac0d31c43f9a36
00319 - Inserting 3 slots for driver_id abae2bc074c8b4af3a7272b12ba66510
00320 - Inserting 14 slots for driver_id e7e229970fddc14df7df0d90f4a8ff0c
00321 - Inserting 24 slots for driver_id fae272b321c99415bc913513053c5fd2
00322 - Inserting 128 slots for driver_id 4bb467a76f80dab73cb736cbb92512f2
00323 - Inserting 9 slots for driver_id 4a4fc0609d5e034e8e35adf86ed4df9b
00324 - Inserting 5 slots for driver_id dc716906fcfcff036b06123fed559d89
00325 - Inserting 17 slots for driver_id 645d712de8945919d8c9baefbe63e116
00326 - Inserting 9 slots for driver_id fdc998e2466a148f45587e2281a26325
00327 - Inserting 27 slots for driver_id ce6c2d142f74a61d4cb8a310f97251fc
00328 - Inserting 85 slots for driver_id 064f8fb3b1c1dc7c6d1799866dba7eec
00329 - Inserting 15 slots for driver_id 7bd624209ab27bb176fa7949da23715e
00330 - Inserting 3 slots for driver_id dc7

00435 - Inserting 55 slots for driver_id 6f7e2476e2ae161cb2b39e836712fa91
00436 - Inserting 5 slots for driver_id abae2bc074c8b4af3a7272b12ba66510
00437 - No available slots found slots for driver_id a6eaa784d041ef2476968e78cf26463a between 2020-03-19 17:37:53 and 2020-03-19 17:39:38
00438 - Inserting 11 slots for driver_id f4f38e02bbf5a7a6b102b81ff04e49ad
00439 - Inserting 44 slots for driver_id 878f28b136310cacb1405a26a29afaf5
00440 - Inserting 20 slots for driver_id 1d1267fffea183eb48fc8cf15c7b3008
00441 - Inserting 18 slots for driver_id 46dff31bb450eccc1b4da5c875399521
00442 - Inserting 8 slots for driver_id a26395e55f7dfbe49c6154b9984f259f
00443 - Inserting 37 slots for driver_id 2f6fe0f9e4b2de4dbdcfa7fed3cd24c4
00444 - Inserting 3 slots for driver_id abae2bc074c8b4af3a7272b12ba66510
00445 - Inserting 7 slots for driver_id 4a4fc0609d5e034e8e35adf86ed4df9b
00446 - Inserting 23 slots for driver_id 71cb6bbdf8fe9de46696e5b273378ecd
00447 - Inserting 22 slots for driver_id 33d29585c0e

...

36542 - No available slots found slots for driver_id fa75f5929c64cc869c1b1f31253ee6bc between 2019-11-06 07:36:53 and 2019-11-06 08:31:05
36543 - Inserting 11 slots for driver_id d9a8d4004053c86b61ec0fb8948f9f9c
36544 - Inserting 14 slots for driver_id 4a9d057569445ba1c96b3d9638aa55c7
36545 - Inserting 6 slots for driver_id 6508a908e5ef2423512101928efdbff2
36546 - Inserting 1 slots for driver_id 73b68ddc2dfcf9219fdc722cc423bf62
36547 - Inserting 6 slots for driver_id de9abe5a3c777ce012c2efef4eec45f0
36548 - Inserting 11 slots for driver_id 73b68ddc2dfcf9219fdc722cc423bf62
36549 - No available slots found slots for driver_id 748d9f2870af84d564c89ec572c685dd between 2019-11-12 15:44:07 and 2019-11-12 16:07:51
36550 - Inserting 6 slots for driver_id c69a4ce5c66bf0bcf9773b329e15766a
36551 - Inserting 6 slots for driver_id c6a8d07bdff354db18051f5df9939f04
36552 - Inserting 6 slots for driver_id fc2ee139394548b08c0a5b1e9b143a94
36553 - Inserting 17 slots for driver_id 0e00603c0c614bbb06fa687

36650 - Inserting 4 slots for driver_id 0afed8f10f7d5390dcd43facb00ed7ad
36651 - Inserting 4 slots for driver_id 5dae3debf85686d4275bf1b32deddebe
36652 - Inserting 6 slots for driver_id 1c18641d1c8569cc9a92932201bc554b
36653 - Inserting 8 slots for driver_id 9f813b66b46605ace95721908de5e0dc
36654 - Inserting 4 slots for driver_id 2fd9129cc1d4c5f6c39be61aecabbe25
36655 - Inserting 9 slots for driver_id 45922e0e393ec185436ca9e7721f4404
36656 - Inserting 4 slots for driver_id f14ba84679ebdc1325564f8d1af65e46
36657 - Inserting 1 slots for driver_id d9a8d4004053c86b61ec0fb8948f9f9c
36658 - Inserting 11 slots for driver_id da131ee6a04b0b3d75aa0bd7e8fd34c6
36659 - Inserting 6 slots for driver_id 0b73003d04a9654f9da0a007f0ffcde8
36660 - Inserting 8 slots for driver_id 7451d7a0471ba942f72583d5ff30ee4d
36661 - Inserting 16 slots for driver_id 7f5dc72eb1da7142c00e78f8be170dd4
36662 - Inserting 6 slots for driver_id 7fd437bf26f428e859bae34dfc2fb321
36663 - Inserting 3 slots for driver_id f7ea43368

36759 - Inserting 12 slots for driver_id 136d895fd93857865732dc2768c542bb
36760 - Inserting 12 slots for driver_id 57c4b5163f64c1b1497f262b38b19512
36761 - No available slots found slots for driver_id 5604c23312d75f1bc9a2b38cd0df8ee7 between 2019-11-15 12:30:37 and 2019-11-15 12:51:53
36762 - Inserting 17 slots for driver_id 54febc122628046efef0fa1c448eb2fe
36763 - Inserting 7 slots for driver_id 4c40b0da408c2f01d102396e037df9d0
36764 - Inserting 13 slots for driver_id 7dd0a70605cd0c59ca2c4ca5991ec75c
36765 - Inserting 5 slots for driver_id 8e862f7cfb9527d1254fadac59ab48bf
36766 - Inserting 6 slots for driver_id de17b15fdd996a1c41ca7c682de30d54
36767 - Inserting 6 slots for driver_id 0cf504f131cb0201443a7bd449519f4a
36768 - Inserting 13 slots for driver_id 99ec90d4a613ad9c55d7bdbfa068c56d
36769 - Inserting 6 slots for driver_id 96489c0209a979db2f7424f849a06bee
36770 - Inserting 11 slots for driver_id b55f67ff50e377ab2d82d3134b2aad87
36771 - Inserting 4 slots for driver_id b9083a44e89c1

36868 - Inserting 6 slots for driver_id 71d1cfbdabb8f60cd1d8fc2a6ff9b186
36869 - Inserting 4 slots for driver_id 124d798c77fdd61a3432ed66d8246f26
36870 - Inserting 5 slots for driver_id 38162421f6285d0b4d06040b1a1ffd9c
36871 - Inserting 6 slots for driver_id 9478cc8d9b3824528e83ab2b7fca7433
36872 - Inserting 3 slots for driver_id 6d7a7aa69f1660f3090440b2b690786d
36873 - Inserting 6 slots for driver_id b9083a44e89c1e5e128ab9dd553b243f
36874 - Inserting 5 slots for driver_id b5a01c8a19940772474a2249414f1707
36875 - Inserting 7 slots for driver_id 5275fd3741fb050649ec2dbee69bf800
36876 - Inserting 4 slots for driver_id 302fcd8f5b41635ec04e6ff424aa16e8
36877 - Inserting 5 slots for driver_id 73b68ddc2dfcf9219fdc722cc423bf62
36878 - Inserting 7 slots for driver_id 0cbb6e3208b4775d4c85d2d197f54687
36879 - Inserting 5 slots for driver_id 8e93d778b26475fd80b24d2050dc7585
36880 - Inserting 11 slots for driver_id 8512dad2b6515895826b6a3a9b70a0d8
36881 - Inserting 14 slots for driver_id a5d88034c

36991 - Inserting 7 slots for driver_id b188c891427612dfef0b62a32c15eb3c
36992 - Inserting 3 slots for driver_id d7e83a709e29c7af7c912864754b55cb
36993 - Inserting 8 slots for driver_id 3e524bf740dc8cfd3f49bd3e96daee6e
36994 - Inserting 6 slots for driver_id 31c6474f656b953062034c64550c72f9
36995 - Inserting 2 slots for driver_id 475cc020a2dc98e1bd061ddfdf162fdc
36996 - No available slots found slots for driver_id 6691aa84a06bb3643aff534c276e3859 between 2019-10-22 11:08:56 and 2019-10-22 11:28:49
36997 - Inserting 3 slots for driver_id 8cd7a32624d99b65b3a5ce8483c64e04
36998 - Inserting 5 slots for driver_id c159bdf38040547f9de667f1252473f4
36999 - Inserting 6 slots for driver_id b6cdb168702d0b16b8b216a6d3700d70
37000 - Inserting 2 slots for driver_id b56aa454af53a028394a424d474c72ed
37001 - Inserting 10 slots for driver_id 12290f81b634aaec8ee8e70474c06c22
37002 - Inserting 5 slots for driver_id 677fda162fddcebc4c3c007ec27e22b7
37003 - Inserting 1 slots for driver_id 302fcd8f5b41635ec0